In [1]:
import keras
from keras.models import Model
from keras.layers import Dense, Input, Reshape, Lambda
from keras.layers.convolutional import Conv2D
from keras.layers.normalization import BatchNormalization
from keras import backend as K
import os

Using TensorFlow backend.


In [3]:
import experiment
import core
import evaluate

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15400992706783082501
]


In [5]:
model_path = '/Users/rabitt/Desktop/multif0_models/'
model_name = 'multif0_exper10_batchin'

In [11]:
from __future__ import print_function
import matplotlib
import matplotlib.pyplot as plt

import csv
import glob
import librosa
import mir_eval
import numpy as np
import os
import pandas
import scipy

import compute_training_data as C

In [6]:
input_shape = (None, None, 6)
inputs = Input(shape=input_shape)

y0 = BatchNormalization()(inputs)
y1 = Conv2D(256, (5, 5), padding='same', activation='relu', name='bendy1')(y0)
y1a = BatchNormalization()(y1)
y2 = Conv2D(16, (70, 3), padding='same', activation='relu', name='harmonics')(y1a)
y2a = BatchNormalization()(y2)
y3 = Conv2D(128, (3, 3), padding='same', activation='relu', name='smoothy1')(y2a)
y3a = BatchNormalization()(y3)
y4 = Conv2D(8, (360, 1), padding='same', activation='relu', name='distribution')(y3a)
y4a = BatchNormalization()(y4)
y5 = Conv2D(1, (1, 1), padding='same', activation='sigmoid', name='squishy')(y4a)
predictions = Lambda(lambda x: K.squeeze(x, axis=3))(y5)

model = Model(inputs=inputs, outputs=predictions)

In [9]:
model.load_weights(os.path.join(model_path, "{}.pkl".format(model_name)))

In [12]:
def score_on_test_set(model, thresh=0.3, save_path='/Users/rabitt/Desktop/'):
    """score a model on all files in a named test set
    """

    # get files for this test set
    test_set_path = os.path.join("../comparisons/mdb_test/")
    test_npy_files = glob.glob(os.path.join(test_set_path, '*.npy'))

    all_scores = []
    for npy_file in sorted(test_npy_files):
        # get input npy file and ground truth label pair
        file_keys = os.path.basename(npy_file).replace('-', '_').split('_')[:2]
        label_file = glob.glob(
            os.path.join(
                test_set_path,
                "{}*{}*.txt".format(file_keys[0], file_keys[1]))
        )[0]

        # generate prediction on numpy file
        predicted_output, input_hcqt = \
            evaluate.get_single_test_prediction(npy_file, model)

        # save plot for first example
        if len(all_scores) == 0:
            plot_save_path = os.path.join(
                save_path,
                "{}_{}_plot_output.pdf".format(file_keys[0], file_keys[1])
            )

            plt.figure(figsize=(15, 15))

            plt.subplot(2, 1, 1)
            plt.imshow(input_hcqt[0, :, :, 1], origin='lower')
            plt.axis('auto')
            plt.colorbar()

            plt.subplot(2, 1, 2)
            plt.imshow(predicted_output, origin='lower')
            plt.axis('auto')
            plt.colorbar()

            plt.savefig(plot_save_path, format='pdf')
            plt.close()

        # save prediction
        np.save(
            os.path.join(
                save_path,
                "{}_{}_prediction.npy".format(file_keys[0], file_keys[1])
            ),
            predicted_output.astype(np.float32)
        )

        # get multif0 output from prediction
        est_times, est_freqs = evaluate.pitch_activations_to_mf0(predicted_output, thresh)

        # save multif0 output
        evaluate.save_multif0_output(
            est_times, est_freqs,
            os.path.join(
                save_path,
                "{}_{}_prediction.txt".format(file_keys[0], file_keys[1])
            )
        )

        # load ground truth labels
        ref_times, ref_freqs = \
            mir_eval.io.load_ragged_time_series(label_file)

        # get multif0 metrics and append
        scores = mir_eval.multipitch.evaluate(
            ref_times, ref_freqs, est_times, est_freqs)
        scores['track'] = '_'.join(file_keys)
        all_scores.append(scores)

    # save scores to data frame
    scores_path = os.path.join(
        save_path, '{}_all_scores.csv'.format(test_set_name)
    )
    score_summary_path = os.path.join(
        save_path, "{}_score_summary.csv".format(test_set_name)
    )
    df = pandas.DataFrame(all_scores)
    df.to_csv(scores_path)
    df.describe().to_csv(score_summary_path)
    print(df.describe())

In [13]:
score_on_test_set(model, thresh=0.3)

ValueError: axes don't match array